# Writing SQL Queries in Python

It's finally time (just one week in) to liberate yourself from running SQL in the shell! 
Now we will move into to cozy iPython Notebook where we will happily play with our data for the next month.

## Python PostgreSQL libraries

To connect from Python to the PostgreSQL database you've installed on your computer, we need a library (database adapter library). For this class we will use [pg8000](https://github.com/mfenniak/pg8000). It is written entirely in Python, and it is easy to install. 

The other library many people use, psycopg2, is much faster(http://blog.fizyk.net.pl/blog/sqlalchemy-speed-tests-on-postgres-and-mysql.html) but tricky to install--so we will stick with the basics. If you find you need greater performance in the future, you will find the coding syntax quite similar. If you want to learn more about these packages [here is a nice summary](https://www.openscg.com/2017/01/how-to-pick-a-postgresql-python-driver/)



## Installing pg8000

First make sure you have `pip` installed on computer! Then write:

    pip3 install pg8000
    
(Hopefully this will do the trick. Use sudo if that's what you do.)

## Connecting to a database with pg8000

When using a SQL server from Python, two main objects are necessary to make the connection and to send queries and commands:

* 1. The *connection Object*, pg8000.connect(), which connects you to the database and server. You can pass a few parameters inside the connect() function, including the name of the database, user, password and others. (See documentation here)[http://pythonhosted.org/pg8000/dbapi.html#pg8000.connect]
* 2. *cursor objects*, which you use to make SQL queries and retrieve data returned from those queries.

To create a connection object, call `pg8000`'s `connect()` function:

In [ ]:
import pg8000
#conn = pg8000.connect(database="mondial")
#You may need to specify the Username
conn = pg8000.connect(database="mondial", user="Jon")
print(type(conn))

## Making a query

Now that we've connected, you need create a cursor() object through which we will send commands. (Imagine this as an actual blinking cursor on page that you will type into. We are using the connection object's `.cursor()` method:

In [ ]:
cursor = conn.cursor()
print(type(cursor))

The cursor object has several methods of interest to us. You can find all of the properties of the cursor method on the [documentation page](http://pythonhosted.org/pg8000/dbapi.html)The first and most important is `.execute()`, which takes a SQL statement (in a Python string) as a parameter:

In [ ]:
#Longest rivers
cursor.execute("SELECT name, max(length) FROM river GROUP BY name ORDER BY max(length) DESC NULLS LAST")

The `.execute()` performs the query, but doesn't return any values. After calling `.execute()`, you can call the cursor's `.fetchone()`, `.fetchmany()` or `.fetchall()`  methods to get rows returned from the query:

In [ ]:
#This gets one row, if you run again it gets the next row
cursor.fetchone()

In [ ]:
#This gets the next 8 rows
cursor.fetchmany(8)

In [ ]:
#This gets the rest of the rows (or a all of the rows if you call it first)
cursor.fetchall()

To get all of the rows returned from a query, you can also use the cursor object in a `for` loop, like so:

In [ ]:
myquery = "SELECT name, max(length) FROM river GROUP BY name ORDER BY max(length) LIMIT 20;"
cursor.execute(myquery)
for row in cursor:
    print(row)

In [ ]:
#Same thing but an extra variable
myquery = "SELECT name, max(length) FROM river GROUP BY name ORDER BY max(length) LIMIT 20;"
cursor.execute(myquery)
myresult = cursor.fetchall()
for row in myresult:
    print(row)

In [ ]:
#Same thing but with space for a big query
myquery = '''
SELECT name, max(length) 
FROM river 
GROUP BY name 
ORDER BY max(length) 
LIMIT 20;
'''
cursor.execute(myquery)
myresult = cursor.fetchall()
for row in myresult:
    print(row)

The `.fetchone()` method gets you a LIST for the results of each row (each row is one element in a TUPLE of LISTS returned by the cursor object.)

## Flexible Queries
Now that we are in Python we can be more flexible about what goes in and out of our queries. For example, if we are getting names of rivers inputted by the users of our website, we can set up a query to get information about the river from the database.

We could build our query as a simple variable (but you will see we don't want to):

In [ ]:
userwants = "Mississippi"
badquery = "SELECT length FROM river WHERE name = '" + userwants + "'";
print(badquery)

In [ ]:
cursor.execute(badquery)
cursor.fetchone()[0]

So why did I name that variable "badquery"? A couple reasons. First, if there was a river with an ' in its name, like "Roger's River" the query would break.

In [ ]:
bad_river = "Roger's River"
query = "SELECT length FROM river WHERE name = '" + bad_river + "'"
print(query)

For this reason and others, the cursor object's `.execute()` method comes with a built-in means of interpolating values into queries. Use `%s` as wildcards in your query string wherever you want to insert a value, and then pass as a second parameter to `.execute()` via list of values that you want to be included in the query:

In [ ]:
cursor.execute("SELECT length FROM river WHERE name = %s",
              ["Roger's River"])

pg8000 deals with interpolated to string properly, and protects you from [SQL injection attacks](https://en.wikipedia.org/wiki/SQL_injection).

Here's an example looping through a list of rivers to get their length:

In [ ]:
user_wants_rivers = ["Zaire","Nile","Missouri","Indus"]
for river_name in user_wants_rivers:
    cursor.execute("SELECT length FROM river WHERE name = %s",
                   [river_name])
    length = cursor.fetchone() # fetchone() returns a tuple w/1 val
    print(river_name, length)

##  errors in pg8000

Simple errors can break your connection to the database.

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECTT length FROM river WHERE name = 'Nile'")

... you'll get a syntax error. But also you will keep getting errors even when you fix the problem:

In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT length FROM river WHERE name = 'Nile'")

The way to fix this problem is to close the connection and re-open it, by calling the connection object's `rollback` method:

In [ ]:
conn.rollback()

Now your queries can proceed as planned:

In [ ]:
cursor.execute("SELECT length FROM river WHERE name = 'Nile'")
cursor.fetchone()

## More information

For more information, go to [pg8000's documentation](http://pythonhosted.org/pg8000/).